<a href="https://colab.research.google.com/github/aditigarg08/Green-Crop-Analyzer/blob/main/Green_crop_Analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# NOTE: The following cells are for Google Colab only


In [ ]:
## Dataset
PlantVillage dataset can be downloaded from Kaggle:
https://www.kaggle.com/datasets/emmarex/plantdisease


In [ ]:
!kaggle datasets download -d emmarex/plantdisease -p /content
!unzip -o /content/plantdisease.zip -d /content/dataset



In [ ]:
!pip uninstall -y tensorflow keras keras-core keras-nightly tf-keras tensorflow-hub tensorflow-text tensorflow-decision-forests jax jaxlib ydf ml-dtypes protobuf flax orbax-checkpoint dopamine-rl keras-hub -q
!pip install tensorflow==2.18.0 numpy==1.26.4 matplotlib seaborn opencv-python-headless --quiet


In [ ]:
import tensorflow as tf
print("TensorFlow:", tf.__version__)
print("Keras:", tf.keras.__version__)
print("GPU:", tf.config.list_physical_devices('GPU'))


In [ ]:
!pip uninstall -y keras keras-core keras-nightly tf-keras
!pip install -U tensorflow==2.18.0 numpy==1.26.4 --quiet


In [ ]:
import tensorflow as tf
print("TensorFlow:", tf.__version__)
print("Keras:", tf.keras.__version__)


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2

print("TensorFlow:", tf.__version__)
print("MobileNetV2 imported successfully!")


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import os, matplotlib.pyplot as plt

DATA_DIR = '/content/dataset/PlantVillage'
BATCH_SIZE = 32
IMG_SIZE = (128,128)
EPOCHS = 10
FINE_TUNE_EPOCHS = 5
AUTOTUNE = tf.data.AUTOTUNE

# Split automatically (80/20)
train_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR, validation_split=0.2, subset='training', seed=123,
    image_size=IMG_SIZE, batch_size=BATCH_SIZE, label_mode='categorical'
)
val_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR, validation_split=0.2, subset='validation', seed=123,
    image_size=IMG_SIZE, batch_size=BATCH_SIZE, label_mode='categorical'
)

class_names = train_ds.class_names
num_classes = len(class_names)
print("Classes:", class_names)

train_ds = train_ds.map(lambda x,y: (preprocess_input(x), y)).cache().prefetch(AUTOTUNE)
val_ds = val_ds.map(lambda x,y: (preprocess_input(x), y)).cache().prefetch(AUTOTUNE)

# Data Augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1)
])

# Base Model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(*IMG_SIZE,3))
base_model.trainable = False

inputs = layers.Input(shape=(*IMG_SIZE,3))
x = data_augmentation(inputs)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(128, activation='relu')(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)
model = models.Model(inputs, outputs)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Train
history = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS)

# Fine-tune last layers
base_model.trainable = True
fine_tune_at = len(base_model.layers) - 30
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
history_fine = model.fit(train_ds, validation_data=val_ds, epochs=FINE_TUNE_EPOCHS)

# Save model to Drive
MODEL_PATH = "models/trained_model_mobilenetv2.keras"
CLASS_NAMES_PATH = "models/class_names.json"
model.save(MODEL_PATH)
print('✅ Model saved to Drive successfully!')


In [ ]:
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'] + history_fine.history['accuracy'])
plt.title('Training Accuracy')
plt.subplot(1,2,2)
plt.plot(history.history['val_accuracy'] + history_fine.history['val_accuracy'])
plt.title('Validation Accuracy')
plt.show()


In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

for fname in uploaded.keys():
    img_path = f'/content/{fname}'
    img = image.load_img(img_path, target_size=IMG_SIZE)
    arr = image.img_to_array(img)
    arr = np.expand_dims(arr, axis=0)
    arr = preprocess_input(arr)

    pred = model.predict(arr)
    idx = np.argmax(pred)

    # Display results
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"{fname} → Prediction: {class_names[idx]}")
    plt.show()



In [ ]:
import json
with open('/content/drive/MyDrive/class_names.json', 'w') as f:
    json.dump(class_names, f)
print('✅ Saved class names to Drive')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# NOTE: The following cells are for Google Colab only


In [ ]:
!pip install streamlit tensorflow pillow pyngrok --quiet


In [ ]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from PIL import Image
import json

# NOTE:
# Download the trained model from Google Drive and place it in /models/
MODEL_PATH = "models/trained_model_mobilenetv2.keras"
CLASS_NAMES_PATH = "class_names.json"

# Load model and class names
@st.cache_resource
def load_model():
    model = tf.keras.models.load_model(MODEL_PATH)
    return model

model = load_model()

with open(CLASS_NAMES_PATH, "r") as f:
    class_names = json.load(f)

IMG_SIZE = (128, 128)

st.title("🌿 Green Crop Analyzer")
st.write("Upload a leaf image to detect plant disease.")

uploaded_files = st.file_uploader("Upload leaf images", type=["jpg", "jpeg", "png"], accept_multiple_files=True)

if uploaded_files:
    for uploaded_file in uploaded_files:
        img = Image.open(uploaded_file)
        st.image(img, caption=f"Uploaded: {uploaded_file.name}", use_column_width=True)

        img = img.resize(IMG_SIZE)
        arr = image.img_to_array(img)
        arr = np.expand_dims(arr, axis=0)
        arr = preprocess_input(arr)

        pred = model.predict(arr)
        idx = np.argmax(pred)

        st.success(f"Prediction: **{class_names[idx]}**")


In [ ]:
!ngrok config add-authtoken ("YOUR_SECRET_TOKEN")


In [ ]:
from pyngrok import ngrok
!streamlit run app.py &>/content/logs.txt &
url = ngrok.connect(8501)
print("Your Web App URL:", url)


In [ ]:
%%writefile .gitignore
# 🔐 Secrets
kaggle.json
*.env
ngrok.yml

# 🧠 Models
*.h5
*.keras
*.ckpt

# 📊 Dataset
dataset/

# 🐍 Python
__pycache__/
.ipynb_checkpoints/